In [ ]:
pip install pandas torch torchvision scikit-learn sentence-transformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import transformers
from sentence_transformers import SentenceTransformer

# Load the data
file_path = "combined.csv"  # Update with your file path
data = pd.read_csv(file_path, delimiter=";", on_bad_lines='skip')

# Extract the relevant columns
texts = data["Summary"].astype(str)
keywords = data["Keywords"].astype(str)
labels = data["područje_rada"].astype(str)

# Encode labels (convert string classes to integers)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split into train and test sets
X_train_summaries, X_test_summaries, X_train_keywords, X_test_keywords, y_train, y_test = train_test_split(
    texts, keywords, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels
)

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

# Load the embedding model
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Generate embeddings for summaries and keywords
train_summary_embeddings = torch.tensor(embedder.encode(X_train_summaries.reset_index(drop=True).tolist(), convert_to_tensor=True))
test_summary_embeddings = torch.tensor(embedder.encode(X_test_summaries.reset_index(drop=True).tolist(), convert_to_tensor=True))

train_keyword_embeddings = torch.tensor(embedder.encode(X_train_keywords.reset_index(drop=True).tolist(), convert_to_tensor=True))
test_keyword_embeddings = torch.tensor(embedder.encode(X_test_keywords.reset_index(drop=True).tolist(), convert_to_tensor=True))

<ipython-input-4-e0f3b25e4a31>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_summary_embeddings = torch.tensor(embedder.encode(X_train_summaries.reset_index(drop=True).tolist(), convert_to_tensor=True))
<ipython-input-4-e0f3b25e4a31>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_summary_embeddings = torch.tensor(embedder.encode(X_test_summaries.reset_index(drop=True).tolist(), convert_to_tensor=True))
<ipython-input-4-e0f3b25e4a31>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_keyword_embeddings = torch.tensor(em

In [ ]:
import torch.nn as nn
import torch.optim as optim

class MultiInputClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(MultiInputClassifier, self).__init__()

        # Separate pathways for summaries and keywords
        self.summary_fc = nn.Linear(input_dim, hidden_dim)
        self.keyword_fc = nn.Linear(input_dim, hidden_dim)

        # Combined pathway
        self.fc_out = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, summary_embeddings, keyword_embeddings):
        summary_out = torch.relu(self.summary_fc(summary_embeddings))
        keyword_out = torch.relu(self.keyword_fc(keyword_embeddings))

        combined = torch.cat((summary_out, keyword_out), dim=1)  # Combine features
        output = self.fc_out(combined)
        return output

# Hyperparameters
input_dim = train_summary_embeddings.shape[1]
hidden_dim = 128
num_classes = len(label_encoder.classes_)

# Initialize the model, loss, and optimizer
model = MultiInputClassifier(input_dim, hidden_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Convert labels to tensors
y_train_tensor = torch.tensor(y_train)
y_test_tensor = torch.tensor(y_test)

# Training loop
epochs = 10
batch_size = 32

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    # Train in mini-batches
    for i in range(0, len(train_summary_embeddings), batch_size):
        batch_summaries = train_summary_embeddings[i:i+batch_size]
        batch_keywords = train_keyword_embeddings[i:i+batch_size]
        batch_labels = y_train_tensor[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(batch_summaries, batch_keywords)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_summary_embeddings):.4f}")


Epoch 1/10, Loss: 0.0447
Epoch 2/10, Loss: 0.0314
Epoch 3/10, Loss: 0.0282
Epoch 4/10, Loss: 0.0265
Epoch 5/10, Loss: 0.0254
Epoch 6/10, Loss: 0.0245
Epoch 7/10, Loss: 0.0238
Epoch 8/10, Loss: 0.0231
Epoch 9/10, Loss: 0.0225
Epoch 10/10, Loss: 0.0220


In [ ]:
from sklearn.metrics import classification_report

model.eval()
with torch.no_grad():
    predictions = model(test_summary_embeddings, test_keyword_embeddings)
    predicted_classes = torch.argmax(predictions, dim=1)

# Decode predicted labels back to class names
predicted_labels = label_encoder.inverse_transform(predicted_classes.numpy())
true_labels = label_encoder.inverse_transform(y_test)

# Print a classification report
print(classification_report(true_labels, predicted_labels))


                         precision    recall  f1-score   support

biomedicina i zdravstvo       0.84      0.79      0.81       400
   biotehničke znanosti       0.68      0.66      0.67       400
     društvene znanosti       0.59      0.72      0.65       400
  humanističke znanosti       0.74      0.65      0.70       400
      prirodne znanosti       0.67      0.67      0.67       400
      tehničke znanosti       0.64      0.63      0.64       400
    umjetničko područje       0.75      0.77      0.76       400

               accuracy                           0.70      2800
              macro avg       0.70      0.70      0.70      2800
           weighted avg       0.70      0.70      0.70      2800



In [ ]:
new_summary = ["Cilj rada je bio prikazati uzgoj i proizvodnju pšenice u Agro – Tovarniku d.o.o. tijekom 2020./2021. vegetacijske sezone i utvrditi utjecaj primijenjene tehnologije uzgoja te vremenskih prilika na prinos i kvalitetu zrna dviju sorti pšenice. Za potrebe analize tehnologije proizvodnje pšenice korišteni su interni podaci tvrtke Agro-Tovarnik, a za analizu vremenskih uvjeta podaci Državnog hidrometeorološkog zavoda s klimatološke postaje Vinkovci. Vegetacijsko razdoblje pšenice 2020./2021. bilo je pogodno za rast i razvoj pšenice, uz relativno povoljne prosječne temperature zraka i oborinski režim, naročito u travnju i svibnju, u vrijeme vlatanja i klasanja pšenice. Sorta Kraljica uzgajana je na 169 ha, a sorta Sofru na 192 ha. Predusjevi pšenici bili su soja, suncokret i kukuruz. Na većini parcela izostavljeno je oranje te je tlo obrađeno tanjuranjem. "]
new_keywords = ["pšenica, sorte, tehnologija uzgoja, prinos, vremenske prilike, kvaliteta zrna"]

# Generate embeddings for the new input
new_summary_embedding = torch.tensor(embedder.encode(new_summary, convert_to_tensor=True))
new_keyword_embedding = torch.tensor(embedder.encode(new_keywords, convert_to_tensor=True))

model.eval()
with torch.no_grad():
    new_prediction = model(new_summary_embedding, new_keyword_embedding)
    predicted_class = torch.argmax(new_prediction).item()
    confidence = torch.softmax(new_prediction, dim=1)[0][predicted_class].item()

predicted_label = label_encoder.inverse_transform([predicted_class])
print(f"Predicted Class: {predicted_label[0]}")
print(f"Confidence: {confidence * 100:.2f}%")


Predicted Class: biotehničke znanosti
Confidence: 96.06%


<ipython-input-8-49aa58b1d30d>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_summary_embedding = torch.tensor(embedder.encode(new_summary, convert_to_tensor=True))
<ipython-input-8-49aa58b1d30d>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_keyword_embedding = torch.tensor(embedder.encode(new_keywords, convert_to_tensor=True))


In [ ]:
scripted_model = torch.jit.script(model)
scripted_model.save("multi_input_classifier.pt")

torch.save(label_encoder, "multi_input_label_encoder.lbl")

torch.save(embedder, "multi_input_embedder.emb")

In [ ]:
import torch
import torch.nn.functional as F

import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")

model =  torch.jit.load("./multi_input_classifier.pt")
#model.eval()
label_encoder = torch.load("./multi_input_label_encoder.lbl")
embedder = torch.load("./multi_input_embedder.emb")

In [ ]:
new_summary = ["Cilj rada je bio prikazati uzgoj i proizvodnju pšenice u Agro – Tovarniku d.o.o. tijekom 2020./2021. vegetacijske sezone i utvrditi utjecaj primijenjene tehnologije uzgoja te vremenskih prilika na prinos i kvalitetu zrna dviju sorti pšenice. Za potrebe analize tehnologije proizvodnje pšenice korišteni su interni podaci tvrtke Agro-Tovarnik, a za analizu vremenskih uvjeta podaci Državnog hidrometeorološkog zavoda s klimatološke postaje Vinkovci. Vegetacijsko razdoblje pšenice 2020./2021. bilo je pogodno za rast i razvoj pšenice, uz relativno povoljne prosječne temperature zraka i oborinski režim, naročito u travnju i svibnju, u vrijeme vlatanja i klasanja pšenice. Sorta Kraljica uzgajana je na 169 ha, a sorta Sofru na 192 ha. Predusjevi pšenici bili su soja, suncokret i kukuruz. Na većini parcela izostavljeno je oranje te je tlo obrađeno tanjuranjem. "]
new_keywords = ["pšenica, sorte, tehnologija uzgoja, prinos, vremenske prilike, kvaliteta zrna"]

# Generate embeddings for the new input
new_summary_embedding = torch.tensor(embedder.encode(new_summary, convert_to_tensor=True))
new_keyword_embedding = torch.tensor(embedder.encode(new_keywords, convert_to_tensor=True))

model.eval()
with torch.no_grad():
  new_prediction = model(new_summary_embedding, new_keyword_embedding)
  probabilities = F.softmax(new_prediction)
  confidence = torch.max(probabilities).item()

  predicted_class = torch.argmax(new_prediction).item()

predicted_label = label_encoder.inverse_transform([predicted_class])
print(f"Predicted Class: {predicted_label[0]}")
print(f"Confidence: {confidence * 100:.2f}%")


Predicted Class: biotehničke znanosti
Confidence: 96.06%
